In [1]:
from aa_lib import *
import pandas as pd
import numpy as np

In [2]:
model_returns = pd.read_pickle('model_returns_final.pkl')
sharpe_df = pd.read_pickle('sharpe_df_final.pkl')

In [4]:
# ef = get_ef(returns=model_returns, annualize=True)



In [ ]:
def plot_ef(ef, ports, title='Efficient Frontier', save=True):
    mpl.style.use('seaborn')
    fig, ax = plt.subplots(1, 1, figsize=(14, 3))
    ax.plot(ef['Volatility'].values, ef['Return'].cummax().values)
    idx = ef['Sharpe'].idxmax()
    cm = plt.get_cmap('gist_rainbow')
    colors = [cm(1.*i/ports.shape[0]) for i in range(ports.shape[0])]
    random.shuffle(colors)
    ax.set_prop_cycle(color=colors)
    legend_list = []
    for r in ports['Risk Group'].unique():
        df = ports.query('`Risk Group` == @r').sort_values('Volatility')
        x_max = [df.loc[df.index[0], 'Volatility']]
        y_max = [df.loc[df.index[0], 'Return']]
        x_min = [df.loc[df.index[-1], 'Volatility']]
        y_min = [df.loc[df.index[-1], 'Return']]
        for i in df.index[1:]:
            if df.loc[i, 'Return'] > y_max[-1]:
                x_max.append(df.loc[i, 'Volatility'])
                y_max.append(df.loc[i, 'Return'])
        for i in df.index[:-1][::-1]:
            if df.loc[i, 'Return'] < y_min[-1]:
                x_min.append(df.loc[i, 'Volatility'])
                y_min.append(df.loc[i, 'Return'])
        ax.fill(x_max+x_min, y_max+y_min, linewidth=3, alpha=.5, label=r)
        for i in df.index:
            ax.scatter(x=df.loc[i, 'Volatility'],
                       y=df.loc[i, 'Return'], label=i)

    ax.scatter(
        ef.loc[idx, 'Volatility'],
        ef.loc[idx, 'Return'],
        marker='*', color='r', s=500, label='Maximum Sharpe ratio')
    plt.title(title)
    plt.legend(labelspacing=0.8, ncol=5, bbox_to_anchor=(1, 0))
    if save:
        fig.savefig('ef.png')


In [ ]:
plot_ef(ef=ef, ports=sharpe_df)

In [ ]:
underlying = pd.DataFrame(index = model_returns.columns, columns = ["Return", "Volatility", "Sharpe"])
underlying.loc[:, "Volatility"] = model_returns.std()*np.sqrt(252)
underlying.loc[:,"Return"] = (1+model_returns).prod()**(252/len(model_returns))-1
underlying.loc[:, "Sharpe"] = underlying.loc[:,"Return"]/underlying.loc[:, "Volatility"]
underlying

In [ ]:
sharpe_df

In [3]:
expected_returns = pd.DataFrame(index=model_returns.columns, columns = ['Returns'], data=[.05,.07, .08, .09, .09, .09, .075, .09, .09])
expected_returns = (1+expected_returns)**(1/252)-1







In [ ]:
model_returns.columns

In [16]:
port_opt = PortfolioOptimization(asset_classes=model_returns.columns, expected_returns=expected_returns, cov=model_returns.cov())
opt_ports = []
for vol in [(.05,.07),(.12,.15),(.15,.17),(.175,.25)]:
    port_opt = PortfolioOptimization(asset_classes=model_returns.columns, expected_returns=expected_returns, cov=model_returns.cov())
    opt_ports.append(port_opt.mean_variance(vol_range=[vol[0]/np.sqrt(252), vol[1]/np.sqrt(252)]))

In [ ]:
opt_ports[0].solve_value

In [5]:
rec = pd.DataFrame(index=['Conservative', 'Balanced', 'Moderately Aggressive','Aggressive'], columns=['Expected Return']+list(model_returns.columns))
rec

,Expected Return,CapPres,Balanced,ProGrowth,AggressiveGrowth,DividendEquity,OppEquity,Notes,Defined,fairway
Conservative,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Balanced,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Moderately Aggressive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aggressive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
for i, port in enumerate(opt_ports):
    rec.iloc[i, 1:] = np.round(port.optimum,2)
    rec.loc[rec.index[i],'Expected Return'] = (1+np.dot(rec.iloc[i, 1:],expected_returns)[0])**252-1
rec

,Expected Return,CapPres,Balanced,ProGrowth,AggressiveGrowth,DividendEquity,OppEquity,Notes,Defined,fairway
Conservative,0.057881,0.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2
Balanced,0.078052,0.28,0.0,0.0,0.08,0.2,0.0,0.04,0.2,0.2
Moderately Aggressive,0.088148,0.05,0.0,0.0,0.31,0.2,0.0,0.05,0.2,0.2
Aggressive,0.09,0.0,0.0,0.0,0.64,0.04,0.15,0.0,0.0,0.17


In [22]:
model_returns.iloc[:,:-1].cov()

,CapPres,Balanced,ProGrowth,AggressiveGrowth,DividendEquity,OppEquity,Notes,Defined
CapPres,0.000017,0.000032,0.000039,0.000044,0.000035,0.000052,0.000026,0.000043
Balanced,0.000032,0.000068,0.000084,0.000099,0.000077,0.000116,0.000061,0.000097
ProGrowth,0.000039,0.000084,0.000105,0.000124,0.000095,0.000145,0.000078,0.000122
AggressiveGrowth,0.000044,0.000099,0.000124,0.000148,0.000112,0.000172,0.000094,0.000145
DividendEquity,0.000035,0.000077,0.000095,0.000112,0.000120,0.000148,0.000070,0.000110
OppEquity,0.000052,0.000116,0.000145,0.000172,0.000148,0.000237,0.000116,0.000169
Notes,0.000026,0.000061,0.000078,0.000094,0.000070,0.000116,0.000318,0.000092
Defined,0.000043,0.000097,0.000122,0.000145,0.000110,0.000169,0.000092,0.000145


In [19]:
port_opt = PortfolioOptimization(asset_classes=model_returns.columns[:-1], 
                                 expected_returns=expected_returns[:-1],
                                 cov=model_returns.iloc[:,:-1].cov())
opt_ports1M = []
for vol in [(.05,.07),(.12,.15),(.15,.17),(.175,.25)]:
    port_opt = PortfolioOptimization(asset_classes=model_returns.columns, expected_returns=expected_returns[:-1], cov=model_returns[:-1].cov())
    opt_ports1M.append(port_opt.mean_variance(vol_range=[vol[0]/np.sqrt(252), vol[1]/np.sqrt(252)]))

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 8 is different from 9)

In [ ]:
rec1m = pd.DataFrame(index=['Conservative', 'Balanced', 'Moderately Aggressive','Aggressive'], columns=['Expected Return']+list(model_returns.columns))
for i, port in enumerate(opt_ports1M):
    rec.iloc[i, 1:] = np.round(port.optimum,2)
    rec1M.loc[rec.index[i],'Expected Return'] = (1+np.dot(rec.iloc[i, 1:],expected_returns)[0])**252-1
rec